In [ ]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os
import copy
import json

In [ ]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [ ]:
keypoint_model = SegmentationModel().eval().cuda()
keypoint_model.load(KeyConfig.trained_model_path)

In [ ]:
image_transform = T.Compose([T.ToTensor()])

In [ ]:
!rm -rf /code/data/datasets/temp_imgs/*


In [ ]:
!rm -rf /code/data/datasets/keypoints/predictions/*

In [ ]:
!rm -rf /code/data/datasets/detection/predictions/*

In [ ]:
!rm -rf /code/data/datasets/common_hall/detection-keypoint-inference/*

In [ ]:
image_paths = list(Config.images_directory.glob("*.jpg"))
random.shuffle(image_paths)
score_threshold: float = 0.5
# image_paths = image_paths[:5]
for image_path in tqdm(image_paths):
    slug = slugify(image_path)
    img = imageio.imread(image_path)
    img_h, img_w, _ = img.shape
    aug_boxes = detection_model(img)
    aug_boxes = aug_boxes[aug_boxes.scores > score_threshold].to_relative().to_tlbr()
    boxes = aug_boxes.array[:,:4]
    keypoints_list = []
    for box in boxes:
        x1, y1, x2, y2 = (box * np.array([img_w, img_h, img_w, img_h])).astype(int)
        roi_w = x2 - x1
        roi_h = y2 - y1
        roi_resized = crop_and_resize(box, img, KeyConfig.image_crop_size[0],KeyConfig.image_crop_size[1])
        roi_resized_w, roi_resized_h = roi_resized.size
        x = image_transform(roi_resized)
        x = torch.unsqueeze(x, 0).cuda()
        keypoints = keypoint_model(x)['keypoints'][0].detach().cpu().numpy()
        keypoints = keypoints_post_inference_processing(
            keypoints, roi_resized_w, roi_resized_h, roi_w, roi_h, x1, y1
        )
        keypoints_list.append(keypoints.astype(float).tolist())
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), thickness=4)
        img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=5, color=(0, 0, 255), thickness=-1) #Blue: Backtire
        img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=5, color=(255, 0, 0), thickness=-1) #Red Front tire
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    save_path = '/code/data/datasets/temp_imgs'
    filename = str(slug) + ".jpg"
    # box_path = Path("/code/data/datasets/detection/predictions")
    cv2.imwrite(os.path.join(save_path, filename), im_rgb)
    aug_boxes.to_darwin_file(
                output_path =  DetConfig.predictions_directory / (slug + ".json"),
                filename = filename,
                label_names = ["vehicle"],
            )
    with open(KeyConfig.predictions_directory / f"{slug}.json", "w") as f:
        f.write(json.dumps(keypoints_list))
        

    # boxes.to_file(get_prediction_path(slug))

In [ ]:
multi_model_path = "/code/data/datasets/common_hall/detection-keypoint-inference"
existing_list = os.listdir("/root/.darwin/datasets/sparrow-computing/kj_speedtrap/releases/allv1/annotations")
detection_annotation_files = os.listdir(DetConfig.predictions_directory)
for detection_annotation_file in detection_annotation_files:
    if detection_annotation_file not in existing_list:
        detection_file = open(DetConfig.predictions_directory/detection_annotation_file)
        detection_annotation_content = json.load(detection_file)
        if (KeyConfig.predictions_directory/detection_annotation_file).exists():
            keypoint_file = open(KeyConfig.predictions_directory/detection_annotation_file)
            keypoint_annotation_content = json.load(keypoint_file)
            # for idx, tire in enumerate(KeyConfig.keypoint_names):
            #     key_dict = {}
            #     key_dict["name"] = tire
            #     key_dict["keypoint"] = {"x": keypoint_annotation_content[idx][0], "y": keypoint_annotation_content[idx][1]}
            #     detection_annotation_content['annotations'].append(key_dict)
            for key_pair in keypoint_annotation_content:
                for idx, tire in enumerate(KeyConfig.keypoint_names):
                    key_dict = {}
                    x, y = key_pair[idx]
                    key_dict["name"] =  tire
                    key_dict["keypoint"] = {
                        "x": x, "y":y
                    }
                    # key_dict["x"] =  x
                    # key_dict["y"] = y
                    detection_annotation_content['annotations'].append(key_dict)
        if len(detection_annotation_content["annotations"]) > 0:
            with open(Path(multi_model_path)/detection_annotation_file, "w") as f:
                f.write(json.dumps(detection_annotation_content))
        else:
            print(detection_annotation_file, "Did not have any predictions")

        

In [ ]:
detection_annotation_content['annotations']